# Workshop: DOCX Scraping in Python  
### Case Study: KDHE Consumer Confidence Reports

## Introduction

In this workshop, we will build a pipeline to extract structured data from Kansas Department of Health and Environment (KDHE) Consumer Confidence Report (CCR) documents.

Public health data is often published as Word/PDF files rather than clean machine-readable tables. Scraping lets us convert these semi-structured documents into analyzable datasets.

## When is scraping useful?

Use scraping when:
- data is publicly available but not downloadable as a single table,
- information is spread across many files/pages,
- repeated document layouts contain fields you can systematically parse.


## Workshop Plan

This workshop focuses on building a document-scraping workflow for KDHE Consumer Confidence Report (CCR) `.docx` files.

### Steps

We will:

1. Inspect the CCR layout and identify where key fields appear (paragraphs vs tables).
2. Review the Python tools used for document parsing and cleaning.
3. Build a scraper that extracts relevant fields from each CCR.
4. Parse and standardize extracted values into a pandas DataFrame.
5. Export a tidy CSV for later analysis.


## Core Python Libraries

- **os**: file paths, folder traversal, and file management  
- **re**: regular expressions for parsing IDs, dates, units, and text patterns  
- **python-docx**: read Word (`.docx`) structure (paragraphs, runs, tables, rows, cells)  
- **pandas**: tabular wrangling, validation, and CSV export  
- **numpy**: helper operations for missing values and numeric transformations  

## Environment

We will work step-by-step in a Jupyter Notebook, so each stage is:

- explained in markdown,
- implemented in code cells,
- and validated with intermediate outputs.


In [178]:
# for colab running the docx package needs to be installed. uncomment and run
# pip install python-docx

In [179]:
from docx import Document
import pandas as pd
import os
import re

In [212]:
# Path to where documents have been saved
basedir = r"\\resfs.home.ku.edu\groups_hipaa\PSYC\kdsc_ClassData\KDSC-CDL-Project2\Data\Full Set of CCR Doc Files" 
subdir = r"ccrs2025" #"ccrs2024docx"
outdir = r"\\resfs.home.ku.edu\groups_hipaa\PSYC\kdsc_ClassData\KDSC-CDL-Project2\Output\CCR_csvs"
root = os.path.join(basedir, subdir)


docx_files = []
for dirpath, dirnames, filenames in os.walk(root):
    for file in filenames:
        # Return only docx files. Ignore temporary docx files
        if file.lower().endswith(".docx") and not file.startswith("~$"):
            full_file = os.path.join(dirpath, file)
            docx_files.append(full_file)

# print(docx_files)
print('first file:', docx_files[0])
doc_path =  docx_files[0]
doc = Document(doc_path)


first file: \\resfs.home.ku.edu\groups_hipaa\PSYC\kdsc_ClassData\KDSC-CDL-Project2\Data\Full Set of CCR Doc Files\ccrs2025\kdhe_A_E\ABBYVILLE-CITY-OF-KS2015512-DOCX.docx


## `python-docx` overview

`python-docx` lets us open and read Word documents as structured Python objects.

For this document-scraping workflow, we are **extracting data only** and building a new DataFrame.  
We do **not** modify source files unless we explicitly call `doc.save(...)`.

### Accessing text in paragraphs

After loading a file (for example, `doc = Document(path)`), paragraph text is available in:

- `doc.paragraphs` → a list of paragraph objects
- `doc.paragraphs[i].text` → the text content of a specific paragraph


We will also be using regular expressions for locating the text within the paragraphs.
More info on regex can be found at https://docs.python.org/3/library/re.html

This is useful when fields that we want to scrape like **PWS Name** and **PWS ID** appear in normal text or headings instead of tables.


In the screenshot we see that paragraph objects are seperate elements whenever a page break is used. Note that a textless paragraph still counts as the index

![Paragraph Image](https://raw.githubusercontent.com/jbodenheimer/KS_BoilWater_Curriculum/main/doc_paragraph_labeled.png)

In [181]:
# Accessing paragraph text example
# Can have small section here for students to answer coding Qs
# How many total paragraphs are in this document? What is the index for paragraph x
doc_path =  docx_files[0]
doc = Document(doc_path)

# 1. Selects one paragraph from the paragraph list.  
paragraphs = doc.paragraphs
p2 = paragraphs[0]
# 2. Reads its raw `.text` content.  
pws_text = p2.text

print("Raw paragraph text idx0:")
print(pws_text.strip())

# 1. Selects one paragraph from the paragraph list.  
paragraphs = doc.paragraphs
p2 = paragraphs[3]
# 2. Reads its raw `.text` content.  
pws_text = p2.text

print("Raw paragraph text idx3:")
print(pws_text.strip())

Raw paragraph text idx0:
CONSUMER CONFIDENCE REPORT CERTIFICATE OF DELIVERY
Raw paragraph text idx3:
PWS NAME:	CITY OF ABBYVILLE			PWS ID: KS2015512


In [182]:
# Accessing paragraph text example cont.

# 1. Selects one paragraph from the paragraph list.  
paragraphs = doc.paragraphs
p2 = paragraphs[3]
# 2. Reads its raw `.text` content.  
pws_text = p2.text

print("Raw paragraph text idx3:")
print(pws_text.strip())

# 3. Uses a regex pattern to extract:
#    - group 1: PWS Name  
#    - group 2: PWS ID 
# Regex captures:
#   group(1) -> text after "PWS Name:" up to "PWS ID:"
#   group(2) -> alphanumeric/hyphen ID after "PWS ID:"
pattern = r"pws\s*name\s*:\s*(.*?)\s+pws\s*id\s*:\s*([A-Za-z0-9\-]+)"

regex_search = re.search(pattern, pws_text, flags=re.IGNORECASE)


# 4. Prints the original paragraph text and the extracted values.
print("Raw paragraph text:")
print(pws_text.strip())


pws_name = regex_search.group(1).strip()
pws_id = regex_search.group(2).strip()

print("\nExtracted values:")
print("PWS Name:", pws_name)
print("PWS ID:", pws_id)


Raw paragraph text idx3:
PWS NAME:	CITY OF ABBYVILLE			PWS ID: KS2015512
Raw paragraph text:
PWS NAME:	CITY OF ABBYVILLE			PWS ID: KS2015512

Extracted values:
PWS Name: CITY OF ABBYVILLE
PWS ID: KS2015512


Above we extracted text from a predefined paragraph (paragraph[3]) but a consideration needs to be made for "will this work with every file?". With large amounts of files it would take too long to check the layout of each file visually, instead we can write code to be robust to small discrepencies in document layout.

To solve this we can compile the text from paragraphs for the whole document and conduct our regex search on the full text, this way if the paragraph location differs between files we will still catch it.

In [183]:
# 1) How many paragraphs?
print("Number of paragraphs:", len(doc.paragraphs))

# 2) Look at the first paragraph
p0 = doc.paragraphs[0]
print("First paragraph text:", p0.text)

# 3) Build one big text block from all paragraph text
all_paragraph_text = ""
for p in doc.paragraphs:
    # print(p.text)                    # optional: show each paragraph
    # all_paragraph_text += p.text # appends each paragraph into one line
    all_paragraph_text += p.text + "\n" # append each paragraph + newline for readability

# Uncomment the following to display all text scraped from the page
# print(all_paragraph_text)

Number of paragraphs: 90
First paragraph text: CONSUMER CONFIDENCE REPORT CERTIFICATE OF DELIVERY


In [184]:
pattern = r"pws\s*name\s*:\s*(.*?)\s+pws\s*id\s*:\s*([A-Za-z0-9\-]+)"

regex_search = re.search(pattern, paragraphs_text, flags=re.IGNORECASE)


# 4. Prints the original paragraph text and the extracted values.
print("Raw paragraph text:")
print(pws_text.strip())


pws_name = regex_search.group(1).strip()
pws_id = regex_search.group(2).strip()

print("\nExtracted values:")
print("PWS Name:", pws_name)
print("PWS ID:", pws_id)

Raw paragraph text:
PWS NAME:	CITY OF ABBYVILLE			PWS ID: KS2015512

Extracted values:
PWS Name: CITY OF ABBYVILLE
PWS ID: KS2015512


### Accessing text within tables

In `python-docx`, tables are stored separately from paragraphs:

- `doc.tables` → list of table objects  
- `tables[i].rows` → row objects for table `i`  
- `tables[i].rows[r].cells[c]` or `tables[i].cell(r, c)` → specific cell  
- `.text` → text content of a cell

This is useful when data are presented in displayed in typical row/column fashion.

> Note: A table that visually continues onto the next page in Word is often still part of the same table object in `python-docx`, even if repeated headers make it look like a new table.


![Table Image](https://raw.githubusercontent.com/jbodenheimer/KS_BoilWater_Curriculum/main/doc_table_labeled.png)

In [185]:
# Accessing table data example
# 1. Loads all tables from the document.
doc_path =  docx_files[0]
doc = Document(doc_path)
tables = doc.tables

print("\nTotal of ", len(tables), "tables found in the document")
# ---need to add pic of the tables and how to iterate over document to get table to use

# 2. Access same cell in two equivalent ways
print("\nFirst cell (cell method):")
print(tables[0].cell(0, 0).text)

# Can also index by calling rows and cells
# print("\nFirst cell (rows/cells indexing):")
# print(tables[0].rows[0].cells[0].text)

# 3. Table dimensions
print("\nRow count in first table:", len(tables[0].rows))
print("Column count in first row:", len(tables[0].rows[0].cells))

# 4. Cleaned text
print("\nFirst cell text:")
print(tables[0].cell(0, 0).text.strip())


Total of  5 tables found in the document

First cell (cell method):
Regulated Contaminants

Row count in first table: 6
Column count in first row: 8

First cell text:
Regulated Contaminants


Like paragraph extraction, table order in a `.docx` file determines index positions.  
If you hard-code `tables[0]`, your script can break if document structure changes between files.

A more robust approach is to loop through all tables and look for a known header (for example, `"regulated contaminants"` in cell `(0, 0)`).


In [186]:
tables = doc.tables
rows_out = []  # Collect parsed row data

for t, table in enumerate(tables):
    # Normalize cell text:
    # - strip() removes leading/trailing whitespace
    # - casefold() makes matching case-insensitive
    header = table.cell(0, 0).text.strip().casefold()

    # Looking for the table in the document regulated contaminants data
    if header == "regulated contaminants":
        # Returns table index t with the text 'regulated contaminants' in cell(0,0)
        print("Matched table index:", t)
        break


Matched table index: 0


## Outline: contaminant-data extraction

These reports include both metadata and water-quality measurements.  
Our goal is to extract a consistent set of fields across documents.

### Core fields to collect
From paragraphs

1. **CCR** Report Year
2. **CCR** Covering Year
3. **PWS Name** (Public Water System name)  
4. **PWS ID** 

From Table

5. **Testing results**, including:
   - Regulated contaminants
   - Lead and copper
   - Chlorine/chloramines (disinfection residual / MRDL-related section)
   - Secondary contaminants (non-health-based standards)
   <!-- - Compliance period (when reported values apply) -->

### Parsing notes

- Some values appear in paragraph text, others in tables, so we will need to use both to extract the desired data.
- Section titles may vary slightly across reports, so pattern matching and defensive parsing of text are important.


## Step 1: Extract PWS metadata from one document (paragraph text)

Before processing many files, we first test extraction on a smaller batch.

In this step we collect the information needed from paragraphs of the document:

1. Open one `.docx` file.
2. Access `doc.paragraphs`.
3. Select the paragraph that contains `PWS Name` and `PWS ID`.
4. Use a regex pattern to extract:
   - `pws name`
   - `pws id`
   - `Covering year`
   - `Calendar year`


In [187]:
# Step 1: Paragraph extraction from ONE document (no file loop)

doc_path = docx_files[0]  # pick one file for demonstration
doc = Document(doc_path)

paragraphs = doc.paragraphs

################# Create Variable with all paragraph text ####################
all_paragraph_text = ""
for p in doc.paragraphs:
    all_paragraph_text += p.text + "\n" # append each paragraph + newline


################# Search for PWS ####################
# This the pws name and pws ID appear in the same line so we can set up a single regex pattern to find both
# We could also accomplish this by searching for pws name and pws ID seperately
pws_pattern = r"pws\s*name\s*:\s*(.*?)\s+pws\s*id\s*:\s*([A-Za-z0-9\-]+)"
match = re.search(pws_pattern, all_paragraph_text, flags=re.IGNORECASE)


if match:
    pws_name = match.group(1).strip()
    pws_id = match.group(2).strip()
    print("\nExtracted:")
    print("pws name:", pws_name)
    print("pws id  :", pws_id)
else:
    pws_name, pws_id = None, None
    print("\nv not found in selected paragraph.")

################# Search for the Reporting Year ####################
CCR_ReportYear_pattern =  r"Consumer Confidence Report\s*[–-]\s*(\d{4}).*?"
CCR_ReportYear_match = re.search(CCR_ReportYear_pattern, all_paragraph_text, flags=re.IGNORECASE)

if CCR_ReportYear_match:
    report_year = int(CCR_ReportYear_match.group(1))
    print("Report year:", report_year)
else:
    report_year = None
    print("\nCCR_ReportYear_pattern not found in selected paragraph.")

################# Search for Calendar Year ####################
CCR_CalYear_pattern = r"Covering Calendar Year\s*[–-]\s*(\d{4})"
CCR_CalYear_match = re.search(CCR_CalYear_pattern, all_paragraph_text, flags=re.IGNORECASE)
if CCR_CalYear_match:
    covering_year = int(CCR_CalYear_match.group(1))
    print("Covering year:", covering_year)
else:
    covering_year = None
    print("\nCCR_CalYear_pattern not found in selected paragraph.")



Extracted:
pws name: CITY OF ABBYVILLE
pws id  : KS2015512
Report year: 2025
Covering year: 2024


## Step 2: Extract regulated contaminant rows from one document (table text)

Now we extract table rows from the same single report.

In this step we:

1. Access `doc.tables`.
2. Find the table whose top-left cell is `"regulated contaminants"`.
3. Read header cells from row 0.
4. Read each data row into a dictionary.
5. Convert extracted rows into a DataFrame.



In [208]:
# Step 2: Table extraction from ONE document

expected_headers = [
    'covering year', 'report_year','pws name', 'pws id', 'regulated contaminants', 'collection date',
    'highest value', 'range\n(low/high)', 'unit', 'mcl', 'mclg', 'typical source'
]

tables = doc.tables
rows_out = []
target_table = None

# Find the regulated contaminants table
for table in tables:

    first_cell = table.cell(0, 0).text.replace('\xa0', ' ').strip().casefold()
    if first_cell == "regulated contaminants":
        target_table = table
        break

if target_table is None:
    print("No 'regulated contaminants' table found.")
    table_df = pd.DataFrame(columns=expected_headers)
else:
    # Clean headers
    headers = [
        cell.text.replace('\xa0', ' ').strip().casefold()
        for cell in target_table.rows[0].cells
    ]

    # Extract data rows (skip header row at index 0)
    for r in range(1, len(target_table.rows)):
        row_data = {
            headers[c]: target_table.cell(r, c).text.replace('\xa0', ' ').strip()
            for c in range(len(headers))
        }
        rows_out.append(row_data)

    table_df = pd.DataFrame(rows_out)

    # Ensure expected columns exist and order them
    for col in expected_headers:
        if col not in table_df.columns:
            table_df[col] = pd.NA
    table_df = table_df[expected_headers]
print(headers)
# print(row_data)
display(table_df)

if "" in headers:
    print("Warning: have blank header(s):", headers)



['regulated contaminants', 'collection date', 'highest value', 'range\n(low/high)', 'unit', '', 'mclg', 'typical source']


,covering year,report_year,pws name,pws id,regulated contaminants,collection date,highest value,range\n(low/high),unit,mcl,mclg,typical source
0,<NA>,<NA>,<NA>,<NA>,ARSENIC,4/15/2024,1.5,1.5,ppb,<NA>,0,Erosion of natural deposits
1,<NA>,<NA>,<NA>,<NA>,BARIUM,4/15/2024,0.46,0.46,ppm,<NA>,2,Discharge from metal refineries
2,<NA>,<NA>,<NA>,<NA>,ETHYLBENZENE,6/10/2024,1.2,0 - 1.2,ppb,<NA>,700,Discharge from petroleum refineries
3,<NA>,<NA>,<NA>,<NA>,NITRATE,4/15/2024,6,5.8 - 6,ppm,<NA>,10,Runoff from fertilizer use
4,<NA>,<NA>,<NA>,<NA>,"XYLENES, TOTAL",6/10/2024,0.013,0.0051 - 0.013,ppm,<NA>,10,Discharge from petroleum factories; Discharge ...


#### Fixing a missing `MCL` header during `.docx` table extraction

Sometimes the `MCL` column ends up empty when extracting the **Regulated Contaminants** table with `python-docx`. This can happen because the header row contains merged cells, and `python-docx` may return an empty string (`""`) for a header cell even though the text is visible in Word.

In our case, the header list looked like:

```python
['regulated contaminants', 'collection date', 'highest value',
 'range\n(low/high)', 'unit', '', 'mclg', 'typical source']


In [209]:
# If there's an empty header between Unit and MCLG, it's MCL
print(headers)
for i in range(1, len(headers)-1):
    if headers[i] == "" and headers[i-1] == "unit" and headers[i+1] == "mclg":
        headers[i] = "mcl"
print(headers)

['regulated contaminants', 'collection date', 'highest value', 'range\n(low/high)', 'unit', '', 'mclg', 'typical source']
['regulated contaminants', 'collection date', 'highest value', 'range\n(low/high)', 'unit', 'mcl', 'mclg', 'typical source']


Putting the header patch directly into the extraction step ensures `MCL` values land in the correct DataFrame column (instead of being stored under a blank column name and later dropped during reordering).


In [205]:
# Step 2: Table extraction from ONE document

expected_headers = [
    'covering year', 'report_year','pws name', 'pws id', 'regulated contaminants', 'collection date',
    'highest value', 'range\n(low/high)', 'unit', 'mcl', 'mclg', 'typical source'
]

tables = doc.tables
rows_out = []
target_table = None

# Find the regulated contaminants table
for table in tables:

    first_cell = table.cell(0, 0).text.replace('\xa0', ' ').strip().casefold()
    if first_cell == "regulated contaminants":
        target_table = table
        break

if target_table is None:
    print("No 'regulated contaminants' table found.")
    table_df = pd.DataFrame(columns=expected_headers)
else:
    # Clean headers
    headers = [
        cell.text.replace('\xa0', ' ').strip().casefold()
        for cell in target_table.rows[0].cells
    ]

    for i in range(1, len(headers) - 1):
        if headers[i] == "" and headers[i-1] == "unit" and headers[i+1] == "mclg":
            headers[i] = "mcl"

    # Extract data rows (skip header row at index 0)
    for r in range(1, len(target_table.rows)):
        row_data = {
            headers[c]: target_table.cell(r, c).text.replace('\xa0', ' ').strip()
            for c in range(len(headers))
        }
        rows_out.append(row_data)

    table_df = pd.DataFrame(rows_out)

    # Ensure expected columns exist and order them
    for col in expected_headers:
        if col not in table_df.columns:
            table_df[col] = pd.NA
    table_df = table_df[expected_headers]
print(headers)
# print(row_data)
display(table_df)

if "" in headers:
    print("Warning: have blank header(s):", headers)



['regulated contaminants', 'collection date', 'highest value', 'range\n(low/high)', 'unit', 'mcl', 'mclg', 'typical source']


,covering year,report_year,pws name,pws id,regulated contaminants,collection date,highest value,range\n(low/high),unit,mcl,mclg,typical source
0,<NA>,<NA>,<NA>,<NA>,BARIUM,1/24/2024,0.16,0.16,ppm,2,2,Discharge from metal refineries
1,<NA>,<NA>,<NA>,<NA>,CHROMIUM,1/24/2024,1.4,1.4,ppb,100,100,Discharge from steel and pulp mills
2,<NA>,<NA>,<NA>,<NA>,FLUORIDE,1/24/2024,0.49,0.49,ppm,4,4,Natural deposits; Water additive which promote...
3,<NA>,<NA>,<NA>,<NA>,NITRATE,1/24/2024,8.4,8 - 8.4,ppm,10,10,Runoff from fertilizer use
4,<NA>,<NA>,<NA>,<NA>,SELENIUM,1/24/2024,1.5,1.5,ppb,50,50,Erosion of natural deposits


## Step 3: Scale up to multiple files

After validating extraction on one report, we can apply the same logic across many documents.

This full-loop version:

1. Iterates through selected files.
2. Extracts `pws name`/`pws id` from paragraphs.
3. Finds and parses the `"regulated contaminants"` table.
4. Appends each file’s rows into one combined DataFrame.


In [ ]:
#Table Extraction for REgulated Contaminants
expected_headers = ['covering year', 'report year', 'pws name', 'pws id', 'regulated contaminants', 'collection date', 'highest value', 'range\n(low/high)','unit','mcl','mclg','typical source']

# Setup a dataframe for extracted data
df = pd.DataFrame(columns=expected_headers) 

# Loops through all files and perform extraction on each file
for i, file in enumerate(docx_files):

    # Index and bring a docx file 
    doc_path = (docx_files[i])

    doc = Document(doc_path)

    ###################### Paragraph Extraction #####################
    paragraphs = doc.paragraphs
    all_paragraph_text = ""
    for p in doc.paragraphs:
        all_paragraph_text += p.text + "\n" # append each paragraph + newline

    # Extract pws name and pws ID
    pws_pattern = r"pws\s*name\s*:\s*(.*?)\s+pws\s*id\s*:\s*([A-Za-z0-9\-]+)"
    match = re.search(pws_pattern, all_paragraph_text, flags=re.IGNORECASE)

    if match:
        pws_name = match.group(1).strip()
        pws_id = match.group(2).strip()
        # print("\nExtracted:")
        # print("pws name:", pws_name)
        # print("pws id  :", pws_id)
    else:
        pws_name, pws_id = None, None
        print("\nv not found in selected paragraph.")

    # Extract the Report Year from the file
    CCR_ReportYear_pattern =  r"Consumer Confidence Report\s*[–-]\s*(\d{4}).*?"
    CCR_ReportYear_match = re.search(CCR_ReportYear_pattern, all_paragraph_text, flags=re.IGNORECASE)

    if CCR_ReportYear_match:
        report_year = int(CCR_ReportYear_match.group(1))
        # print("Report year:", report_year)
    else:
        report_year = None
        print("\nCCR_ReportYear_pattern not found in selected paragraph.")

    # Extract the Calendar Year from the file
    CCR_CalYear_pattern = r"Covering Calendar Year\s*[–-]\s*(\d{4})"
    CCR_CalYear_match = re.search(CCR_CalYear_pattern, all_paragraph_text, flags=re.IGNORECASE)
    if CCR_CalYear_match:
        covering_year = int(CCR_CalYear_match.group(1))
        # print("Covering year:", covering_year)
    else:
        covering_year = None
        print("\nCCR_CalYear_pattern not found in selected paragraph.")

    
    ###################### Table Extraction #########################
    tables = doc.tables
    rows_out = [] # setup variable for row data
    for t, table in enumerate(tables):       

        if tables[t].cell(0,0).text.strip().casefold() == "regulated contaminants": #strip removes white space and casefold avoids capitalization issues
            txt = table.cell(0, 5).text
            raw = table.cell(0, 5).text
            clean = raw.replace('\xa0', ' ').strip()

            headers = [cell.text.strip().casefold() for cell in table.rows[0].cells] #get all the headers of the current table

            # If there's an empty header between Unit and MCLG, it's MCL
            for i in range(1, len(headers)-1):
                if headers[i] == "" and headers[i-1] == "unit" and headers[i+1] == "mclg":
                    headers[i] = "mcl"
            if "" in headers:
                print("Warning: have blank header(s):", headers)
            for r in range(1, len(table.rows)):
                # Explain 
                row_data = {headers[c]: table.cell(r, c).text.strip() for c in range(len(headers))}
                # print(row_data)
                rows_out.append(row_data)

    # Create a dataframe of the extracted table data
    table_df = pd.DataFrame(rows_out)

    # Add the data extracted from paragraphs to our table dataframe
    table_df["covering year"] = covering_year
    table_df["report year"] = report_year
    table_df["pws name"] = pws_name
    table_df["pws id"] = pws_id

    # Append 
    df = pd.concat([df, table_df], ignore_index=True)

display(df)



,covering year,report year,pws name,pws id,regulated contaminants,collection date,highest value,range\n(low/high),unit,mcl,mclg,typical source,water system
0,2024,2025,CITY OF ABBYVILLE,KS2015512,BARIUM,1/24/2024,0.16,0.16,ppm,2,2,Discharge from metal refineries,NaN
1,2024,2025,CITY OF ABBYVILLE,KS2015512,CHROMIUM,1/24/2024,1.4,1.4,ppb,100,100,Discharge from steel and pulp mills,NaN
2,2024,2025,CITY OF ABBYVILLE,KS2015512,FLUORIDE,1/24/2024,0.49,0.49,ppm,4,4,Natural deposits; Water additive which promote...,NaN
3,2024,2025,CITY OF ABBYVILLE,KS2015512,NITRATE,1/24/2024,8.4,8 - 8.4,ppm,10,10,Runoff from fertilizer use,NaN
4,2024,2025,CITY OF ABBYVILLE,KS2015512,SELENIUM,1/24/2024,1.5,1.5,ppb,50,50,Erosion of natural deposits,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3766,2024,2025,CITY OF ZENDA,KS2009506,ARSENIC,4/15/2024,1.5,1.5,ppb,10,0,Erosion of natural deposits,NaN
3767,2024,2025,CITY OF ZENDA,KS2009506,BARIUM,4/15/2024,0.46,0.46,ppm,2,2,Discharge from metal refineries,NaN
3768,2024,2025,CITY OF ZENDA,KS2009506,ETHYLBENZENE,6/10/2024,1.2,0 - 1.2,ppb,700,700,Discharge from petroleum refineries,NaN
3769,2024,2025,CITY OF ZENDA,KS2009506,NITRATE,4/15/2024,6,5.8 - 6,ppm,10,10,Runoff from fertilizer use,NaN


In [213]:
# Save Dataframe to the outpur dir as a csv with the directory name we got the files from
# df.to_csv(outdir, '/', subdir + "regulated_contaminants_output.csv", index=False)
out_path = os.path.join(outdir, f"{subdir}_regulated_contaminants_output.csv")
df.to_csv(out_path, index=False)

## Data Quality & Exploration Questions

Data quality and summary questions for the CCR created table.

---

### Question 1) Coverage and uniqueness
- **How many unique water systems are represented in the table?**
  - Use: `pws id`
  - Suggested method: `.nunique()`

---

### Question 2) Missing values
- **How many rows are missing one or more fields? Which headers have missing data?**
  - Suggested method: `.isna()` + row-wise checks

---

### Question 3) Year consistency checks
- **Are `report year` and `covering year` ever inconsistent?**
  - Identify rows where the two year fields do not align with expected logic.
  - Suggested method: `.value_counts()`

---

### Question 4) Duplicate observation checks
- **Do duplicate observations exist for the same observation?**
  - Suggested method: `.duplicated(...)`

---

### Question 5) Contaminant inventory and reporting breadth
- **What contaminants are reported in the dataset?**
- **How many cities report each contaminant?**
  - Suggested methods:
    - `.value_counts()` for frequency
    - `.nunique()` for city counts

---

### Question 6) Extremes for top contaminants
- **For the top 3 most reported contaminants:**
  - Which city/cities have the **highest** recorded `highest value`?
  - Which city/cities have the **lowest** recorded `highest value`?

---

### Helpful pandas tools
- `.nunique()`
- `.groupby()`
- `.value_counts()`
- `.duplicated()`
- `.isna()`
- `.agg()`
